## Criação da partições de competition

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df_competition = spark.read.option("multiline", "true").json("s3://sor/football/data/competitions.json")

24/01/09 01:51:37 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [79]:
df_competition.show()
df_competition.count()

+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|competition_gender|competition_id|competition_international|competition_name|competition_youth|country_name|     match_available|match_available_360|       match_updated|   match_updated_360|season_id|season_name|
+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|              male|             9|                    false|   1. Bundesliga|            false|     Germany|2023-12-12T07:43:...|               null|2023-12-12T07:43:...|                null|       27|  2015/2016|
|              male|            16|                    false|Champions League|            false|      Europe|2023-03-07T12:20:...|          

70

In [70]:
# Selecionar temporadas distintas
df_competition.select("season_id").distinct().orderBy("season_id").count()

45

In [50]:
partitions = df_competition.select('season_id').distinct().orderBy("season_id").collect()
print(partitions)

[Row(season_id=1), Row(season_id=2), Row(season_id=3), Row(season_id=4), Row(season_id=21), Row(season_id=22), Row(season_id=23), Row(season_id=24), Row(season_id=25), Row(season_id=26), Row(season_id=27), Row(season_id=30), Row(season_id=37), Row(season_id=38), Row(season_id=39), Row(season_id=40), Row(season_id=41), Row(season_id=42), Row(season_id=43), Row(season_id=44), Row(season_id=48), Row(season_id=51), Row(season_id=54), Row(season_id=55), Row(season_id=68), Row(season_id=71), Row(season_id=75), Row(season_id=76), Row(season_id=84), Row(season_id=86), Row(season_id=90), Row(season_id=106), Row(season_id=107), Row(season_id=108), Row(season_id=235), Row(season_id=268), Row(season_id=269), Row(season_id=270), Row(season_id=272), Row(season_id=274), Row(season_id=275), Row(season_id=276), Row(season_id=277), Row(season_id=278), Row(season_id=279)]


In [62]:
dataframes = {}
for partition in partitions:
    partition_value = partition['season_id']
    partition_competition = df_competition.filter(df_competition['season_id'] == partition_value)
    dataframes[partition_value] = partition_competition

In [73]:
# Caminho base no S3
base_path = "s3://landing/"

# Iterar sobre os DataFrames e salvá-los no S3
for partition_value, dataframe in dataframes.items():
    path = f"{base_path}/competitions/season={partition_value}"
    dataframe.write.json(path, mode='overwrite')

In [78]:
read_df = spark.read.json(f"{base_path}/competitions")
read_df.show()
read_df.count()

+------------------+--------------+-------------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+
|competition_gender|competition_id|competition_international|    competition_name|competition_youth|        country_name|     match_available| match_available_360|       match_updated|   match_updated_360|season_id|season_name|season|
+------------------+--------------+-------------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+-----------+------+
|              male|             9|                    false|       1. Bundesliga|            false|             Germany|2023-12-12T07:43:...|                null|2023-12-12T07:43:...|                null|       27|  2015/2016|    27|
|              male|            16|                    false

70

## Criação das partições de Matches

In [8]:

# obs Isso aqui é um paliativo. O correto é ser automatizado.
# Criação da lista manual de competicoes e jogos

lista_competitions = [2,7,9,11,12,16,35,37,43,44,49,53,55,72,81,87,116,1238,1470]
matches_competitions = ["2/27", "2/44", "7/27", "7/108", "7/235", "9/27",
                        "11/1", "11/2", "11/4", "11/21", "11/22", "11/23", "11/24", "11/25", "11/26",
                        "11/27", "11/37", "11/38", "11/39", "11/40", "11/41", "11/42", "11/90", "11/278",
                        "12/27","12/86", "16/1", "16/2", "16/4", "16/21", "16/22", "16/23", "16/24", "16/25",
                        "16/26", "16/27", "16/37", "16/39", "16/41", "16/44", "16/71", "16/76", "16/276", "16/277",
                        "35/75","37/4", "37/42", "37/90", "43/3", "43/51", "43/54", "43/55", "43/106", "43/269", "43/270" ,"43/272",
                        "44/107", "49/3", "53/106", "55/43", "72/30", "72/107", "81/48", "81/275", "87/84", "87/268", "87/279", "116/68",
                        "1238/108", "1470/274"]



In [15]:
# Percorrer todos os arquivos e appendar todos os registros de partidas num só dataframe

first = 1

for competition_match in matches_competitions:
    
    df_match_competition = spark.read.option("multiline", "true").json(f"s3://sor/football/data/matches/{competition_match}.json")
    print(df_match_competition.count())
    if first == 1:
        df = df_match_competition
    else:
        if df_match_competition.count() > 0:
            df.union(df_match_competition)
    first = 0
#df_match_competition.show()

380
38
0
26
32
306
36
34
34
35
33
37
32
31
38
380
7
17
26
28
31
33
35
1


AnalysisException: Union can only be performed on tables with the same number of columns, but the first table has 18 columns and the second table has 17 columns;
'Union false, false
:- Relation [away_score#17089L,away_team#17090,competition#17091,competition_stage#17092,home_score#17093L,home_team#17094,kick_off#17095,last_updated#17096,last_updated_360#17097,match_date#17098,match_id#17099L,match_status#17100,match_status_360#17101,match_week#17102L,metadata#17103,referee#17104,season#17105,stadium#17106] json
+- Relation [away_score#19258L,away_team#19259,competition#19260,competition_stage#19261,home_score#19262L,home_team#19263,kick_off#19264,last_updated#19265,last_updated_360#19266,match_date#19267,match_id#19268L,match_status#19269,match_status_360#19270,match_week#19271L,metadata#19272,season#19273,stadium#19274] json


In [5]:
df_matches_competition27 = spark.read.option("multiline", "true").json("s3://sor/football/data/matches/2/27.json")
df_matches_competition27.show()

+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+--------------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+
|away_score|           away_team|         competition|  competition_stage|home_score|           home_team|    kick_off|        last_updated|    last_updated_360|match_date|match_id|match_status|match_status_360|match_week|     metadata|             referee|         season|             stadium|
+----------+--------------------+--------------------+-------------------+----------+--------------------+------------+--------------------+--------------------+----------+--------+------------+----------------+----------+-------------+--------------------+---------------+--------------------+
|         0|{male, null, 28, ...|{2, Premier Leagu...|{1, Regular Season}|         0|{{68, England}, m...|16:00:00.

In [6]:
competition_id = "2"

colunas_particao = [competition_id, "season.season_id"]

In [7]:
base_path = "s3://landing/"

df_matches_competition27.write \
    .partitionBy(*colunas_particao) \
    .json(f"{base_path}/matches/{competition_id}/27.json")

AnalysisException: Partition column `2` not found in schema struct<away_score:bigint,away_team:struct<away_team_gender:string,away_team_group:string,away_team_id:bigint,away_team_name:string,country:struct<id:bigint,name:string>,managers:array<struct<country:struct<id:bigint,name:string>,dob:string,id:bigint,name:string,nickname:string>>>,competition:struct<competition_id:bigint,competition_name:string,country_name:string>,competition_stage:struct<id:bigint,name:string>,home_score:bigint,home_team:struct<country:struct<id:bigint,name:string>,home_team_gender:string,home_team_group:string,home_team_id:bigint,home_team_name:string,managers:array<struct<country:struct<id:bigint,name:string>,dob:string,id:bigint,name:string,nickname:string>>>,kick_off:string,last_updated:string,last_updated_360:string,match_date:string,match_id:bigint,match_status:string,match_status_360:string,match_week:bigint,metadata:struct<data_version:string,shot_fidelity_version:string,xy_fidelity_version:string>,re

In [ ]:
# Percorrer todos os jogos de todas as competições